In [1]:
from os import listdir
import pandas as pd
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input
from keras.applications.inception_v3 import decode_predictions
from keras.preprocessing import image

Using TensorFlow backend.


In [2]:
import tensorflow as tf

In [4]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
import keras
from keras.callbacks.callbacks import ModelCheckpoint, EarlyStopping
import numpy as np
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras import layers
from keras.models import Model, Sequential

In [6]:
import ijson
import os
# setting root directory to our dataset location
os.chdir('/home/anish_saha/imt575/annotations/')
#os.listdir('/home/anish_saha/imt575/annotations')

Loading the images and its metadata from the captions_train2017 dataset

In [7]:
filename = "captions_train2017.json"
with open(filename, 'r') as f:
    images = list(ijson.items(f, 'images.item'))

In [8]:
images[0]

{'license': 3,
 'file_name': '000000391895.jpg',
 'coco_url': 'http://images.cocodataset.org/train2017/000000391895.jpg',
 'height': 360,
 'width': 640,
 'date_captured': '2013-11-14 11:18:45',
 'flickr_url': 'http://farm9.staticflickr.com/8186/8119368305_4e622c8349_z.jpg',
 'id': 391895}

Counting the number of observations

In [9]:
print("There are", len(images), "images in the MS COCO 2017 training dataset")

There are 118287 images in the MS COCO 2017 training dataset


Loading the annotations for each image ( identified by its unique image id )

In [10]:
with open(filename, 'r') as f:
    annotations = list(ijson.items(f, 'annotations.item'))

In [11]:
annotations[:5]

[{'image_id': 203564,
  'id': 37,
  'caption': 'A bicycle replica with a clock as the front wheel.'},
 {'image_id': 322141,
  'id': 49,
  'caption': 'A room with blue walls and a white sink and door.'},
 {'image_id': 16977,
  'id': 89,
  'caption': 'A car that seems to be parked illegally behind a legally parked car'},
 {'image_id': 106140,
  'id': 98,
  'caption': 'A large passenger airplane flying through the air.'},
 {'image_id': 106140,
  'id': 101,
  'caption': 'There is a GOL plane taking off in a partly cloudy sky.'}]

Below, we print captions for image id 391895

In [12]:
list(o for o in annotations if o['image_id'] == 391895)

[{'image_id': 391895,
  'id': 770337,
  'caption': 'A man with a red helmet on a small moped on a dirt road. '},
 {'image_id': 391895,
  'id': 771687,
  'caption': 'Man riding a motor bike on a dirt road on the countryside.'},
 {'image_id': 391895,
  'id': 772707,
  'caption': 'A man riding on the back of a motorcycle.'},
 {'image_id': 391895,
  'id': 776154,
  'caption': 'A dirt path with a young person on a motor bike rests to the foreground of a verdant area with a bridge and a background of cloud-wreathed mountains. '},
 {'image_id': 391895,
  'id': 781998,
  'caption': 'A man in a red shirt and a red hat is on a motorcycle on a hill side.'}]

In [13]:
print("There are", len(annotations), "captions in our training dataset")

There are 591753 captions in our training dataset


We can see that there are more captions than images which indicate that We can see that each image has many captions associated with it.

#### Check for missing values by finding if each image has at least one caption associated with it 

In [14]:
annotations[0]

{'image_id': 203564,
 'id': 37,
 'caption': 'A bicycle replica with a clock as the front wheel.'}

In [ ]:
def check_missing_values(images, annotations):
    
    image_ids = get_image_id(annotations)
    
    missings = []
    
    for i in range(0, len(images)):
        
        if images[i]['id'] in image_ids:
            
            continue
            
        else:
            
            missings.append(images[i]['id'])
    
    return missings

check_missing_values(images, annotations)

Thus we can see that there are no missing values for our data

In [16]:
#Trying out a method to add start and end seq in the description list creation statement itself


ids = [i['id'] for i in images]

In [17]:
trainids = ids[:50000]

In [18]:
import time

In [19]:
# final
start = time.time()
descriptions = dict()
levels = [num for num in range(0,50000,1000)]
for j in range(len(trainids)):
    if j in levels:
        print(j,"out of",len(trainids),"images are completed")
    i = trainids[j]
    descriptions[i]=list(('startseq ' + o['caption'].replace(".",' endseq')).strip() for o in annotations if o['image_id'] == i)
end = time.time()
print("Total time taken is:",end - start)

0 out of 50000 images are completed
1000 out of 50000 images are completed
2000 out of 50000 images are completed
3000 out of 50000 images are completed
4000 out of 50000 images are completed
5000 out of 50000 images are completed
6000 out of 50000 images are completed
7000 out of 50000 images are completed
8000 out of 50000 images are completed
9000 out of 50000 images are completed
10000 out of 50000 images are completed
11000 out of 50000 images are completed
12000 out of 50000 images are completed
13000 out of 50000 images are completed
14000 out of 50000 images are completed
15000 out of 50000 images are completed
16000 out of 50000 images are completed
17000 out of 50000 images are completed
18000 out of 50000 images are completed
19000 out of 50000 images are completed
20000 out of 50000 images are completed
21000 out of 50000 images are completed
22000 out of 50000 images are completed
23000 out of 50000 images are completed
24000 out of 50000 images are completed
25000 out of 

In [20]:
#ids = [i['id'] for i in images]
#descriptions = dict()
#for i in ids:
#    descriptions[i] = list(o['caption'] for o in annotations if o['image_id'] == i)

In [21]:
for key, desc_list in descriptions.items():
    for i in range(len(desc_list)):
        desc = desc_list[i]
        desc = desc.split()
        desc = [word.lower() for word in desc]
        desc = [word for word in desc if len(word)>1]
        desc = [word for word in desc if word.isalpha()]
        desc_list[i] = ' '.join(desc)

In [22]:
descriptions[391895]

['startseq man with red helmet on small moped on dirt road endseq',
 'startseq man riding motor bike on dirt road on the countryside endseq',
 'startseq man riding on the back of motorcycle endseq',
 'startseq dirt path with young person on motor bike rests to the foreground of verdant area with bridge and background of mountains endseq',
 'startseq man in red shirt and red hat is on motorcycle on hill side endseq']

In [23]:
vocabulary = set()
for key in descriptions.keys():
    [vocabulary.update(d.split()) for d in descriptions[key]]
print('Original Vocabulary Size: %d' % len(vocabulary))

Original Vocabulary Size: 18311


In [24]:
# Consider only words which occur at least 10 times in the corpus
word_count_threshold = 10
word_counts = {}
nsents = 0
for sent in vocabulary:
    nsents += 1
    for w in sent.split(' '):
        word_counts[w] = word_counts.get(w, 0) + 1

vocab = [w for w in word_counts if word_counts[w] >= word_count_threshold]

print('preprocessed words %d ' % len(vocab))

preprocessed words 0 


In [25]:
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input
from keras.applications.inception_v3 import decode_predictions
from keras.preprocessing import image
import numpy as np

In [28]:
from keras.models import Model

# Get the InceptionV3 model trained on imagenet data
model = InceptionV3(weights='imagenet', include_top=True)

# Remove the last layer (output softmax layer) from the inception v3
model_new = Model(model.input, model.layers[-2].output)

In [29]:
len(trainids)

50000

In [30]:
imglist = [i.get('file_name') for i in images for j in trainids if i.get('id')==j]

In [31]:
imglist[:5]

['000000391895.jpg',
 '000000522418.jpg',
 '000000184613.jpg',
 '000000318219.jpg',
 '000000554625.jpg']

In [32]:
from keras.preprocessing.image import load_img
from PIL import Image

In [33]:
from IPython.display import display
from PIL import Image

In [34]:
photos= dict()

for i in range(len(imglist)):
    img_path = '/home/anish_saha/imt575/train2017/'+ imglist[i]

    # Convert all the images to size 299x299 as expected by the inception v3 model
    img = image.load_img(img_path, target_size=(299, 299))

    # Convert PIL image to numpy array of 3-dimensions
    x = image.img_to_array(img)

    # Add one more dimension
    x = np.expand_dims(x, axis = 0)

    # preprocess images using preprocess_input() from inception module
    y = preprocess_input(x)
    
    image_vector = model_new.predict(y)

    #print the numpy array output flatten layer
    ix = int(imglist[i].split('.')[0])
    photos[ix] = image_vector.reshape(-1)

In [35]:
# convert a dictionary of clean descriptions to a list of descriptions
def to_lines(descriptions):
    all_desc = list()
    for key in descriptions.keys():
        [all_desc.append(d) for d in descriptions[key]]
    return all_desc
# calculate the length of the description with the most words
def max_length(descriptions):
    lines = to_lines(descriptions)
    return max(len(d.split()) for d in lines)
# determine the maximum sequence length
max_length = max_length(descriptions)
print('Max Description Length: %d' % max_length)

Max Description Length: 46


In [36]:
ixtoword = {}
wordtoix = {}
ix = 0
for w in vocabulary:
    wordtoix[w] = ix
    ixtoword[ix] = w
    ix += 1
print(wordtoix.items())

dict_items([('alps', 0), ('onlooker', 1), ('roof', 2), ('manhole', 3), ('pensively', 4), ('shrine', 5), ('ballons', 6), ('toast', 7), ('fetlocks', 8), ('cheeseteak', 9), ('convey', 10), ('cow', 11), ('seemingly', 12), ('dozen', 13), ('elf', 14), ('interactions', 15), ('titling', 16), ('dissolve', 17), ('strand', 18), ('maneuvering', 19), ('crumple', 20), ('properly', 21), ('paddleboarder', 22), ('carraige', 23), ('arpir', 24), ('redhaired', 25), ('developmentally', 26), ('crumble', 27), ('constructional', 28), ('lasso', 29), ('chateau', 30), ('lumia', 31), ('chops', 32), ('thatched', 33), ('benefits', 34), ('cattle', 35), ('wiping', 36), ('josset', 37), ('grant', 38), ('contrasts', 39), ('purse', 40), ('amazement', 41), ('literally', 42), ('llama', 43), ('deckers', 44), ('experienced', 45), ('tornado', 46), ('teach', 47), ('hone', 48), ('monior', 49), ('swiftly', 50), ('towel', 51), ('handheld', 52), ('motorcycyle', 53), ('moguls', 54), ('gulliver', 55), ('gainz', 56), ('constitute', 5

In [37]:
# data generator, intended to be used in a call to model.fit_generator()
def data_generator(descriptions, photos, wordtoix, max_length, num_photos_per_batch):
    X1, X2, y = list(), list(), list()
    n=0
    # loop for ever over images
    while 1:
        for key, desc_list in descriptions.items():
            n+=1
            # retrieve the photo feature
            photo = photos[key]
            for desc in desc_list:
                # encode the sequence
                seq = [wordtoix[word] for word in desc.split(' ') if word in wordtoix]
                #print(len(seq))
                # split one sequence into multiple X, y pairs
                for i in range(1, len(seq)):
                    # split into input and output pair
                    in_seq, out_seq = seq[:i], seq[i]
                    # pad input sequence
                    in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
                    #print("Inseq len: "+ str(len(in_seq)))
                    # encode output sequence
                    out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
                    # store
                    #print("outseq len: "+ str(len(out_seq)))
                    X1.append(photo)
                    X2.append(in_seq)
                    y.append(out_seq)
            # yield the batch data
            if n==num_photos_per_batch:
                yield [[np.array(X1), np.array(X2)], np.array(y)]
                X1, X2, y = list(), list(), list()
                n=0

In [11]:
import os
# Load Glove vectors
#download the glove 200 length vector from https://nlp.stanford.edu/projects/glove/
#select the glove 6B value
glove_dir = 'D:/IMT575/Project/glove.6B/'
embeddings_index = {} # empty dictionary
f = open(os.path.join(glove_dir, 'glove.6B.200d.txt'), encoding="utf-8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

In [12]:
embedding_dim = 200
vocab_size = 18311#len(vocabulary)
# Get 200-dim dense vector for each of the 10000 words in out vocabulary
embedding_matrix = np.zeros((vocab_size, embedding_dim))
for word, i in wordtoix.items():
    #if i < max_words:
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in the embedding index will be all zeros
        embedding_matrix[i-1] = embedding_vector

NameError: name 'wordtoix' is not defined

In [ ]:
from keras.layers import Input
from keras.layers import Dense, Softmax, Dropout,Embedding,LSTM,add
# image feature extractor model
inputs1 = Input(shape=(2048,))
fe1 = Dropout(0.5)(inputs1)
fe2 = Dense(256, activation='relu')(fe1)

# partial caption sequence model
inputs2 = Input(shape=(46,))
se1 = Embedding(18311, 200, mask_zero=True)(inputs2)
se2 = Dropout(0.5)(se1)
se3 = LSTM(256)(se2)

# decoder (feed forward) model
decoder1 = add([fe2, se3])
decoder2 = Dense(256, activation='relu')(decoder1)
outputs = Dense(18311, activation='softmax')(decoder2)

# merge the two input models
model = Model(inputs=[inputs1, inputs2], outputs=outputs)
model.summary()

In [13]:
model.layers[2].set_weights([embedding_matrix])
model.layers[2].trainable = False
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=["accuracy"])

In [42]:
# history = model.fit_generator(data_generator(descriptions, photos, wordtoix, max_length=24, num_photos_per_batch=5),steps_per_epoch=5,epochs=30)

# Start of Validation Generator

In [43]:
# load val image data
filename = "captions_val2017.json"
with open(filename, 'r') as f:
    val_images = list(ijson.items(f, 'images.item'))

In [44]:
# val annotations
with open(filename, 'r') as f:
    val_annotations = list(ijson.items(f, 'annotations.item'))

In [45]:
#Trying out a method to add start and end seq in the description list creation statement itself

val_ids = [i['id'] for i in val_images]

In [46]:
# final val descriptions
start = time.time()
val_descriptions = dict()
levels = [num for num in range(0,5000,1000)]
for j in range(len(val_ids)):
    if j in levels:
        print(j,"out of",len(val_ids),"images are completed")
    i = val_ids[j]
    val_descriptions[i]=list(('startseq ' + o['caption'].replace(".",' endseq')).strip() for o in val_annotations if o['image_id'] == i)
end = time.time()
print("Total time taken is:",end - start)

0 out of 5000 images are completed
1000 out of 5000 images are completed
2000 out of 5000 images are completed
3000 out of 5000 images are completed
4000 out of 5000 images are completed
Total time taken is: 8.139778137207031


In [47]:
for key, desc_list in val_descriptions.items():
    for i in range(len(desc_list)):
        desc = desc_list[i]
        desc = desc.split()
        desc = [word.lower() for word in desc]
        desc = [word for word in desc if len(word)>1]
        desc = [word for word in desc if word.isalpha()]
        desc_list[i] = ' '.join(desc)

In [48]:
val_descriptions[397133]

['startseq man is in kitchen making pizzas endseq',
 'startseq man in apron standing on front of oven with pans and bakeware',
 'startseq baker is working in the kitchen rolling dough endseq',
 'startseq person standing by stove in kitchen endseq',
 'startseq table with pies being made and person standing near wall with pots and pans hanging on the wall endseq']

In [49]:
val_imglist = [i.get('file_name') for i in val_images for j in val_ids if i.get('id')==j]

In [50]:
# get val photos
val_photos= {}
for i in range(len(val_imglist)):
    img_path = '/home/anish_saha/imt575/val2017/'+ val_imglist[i]

    # Convert all the images to size 299x299 as expected by the inception v3 model
    img = image.load_img(img_path, target_size=(299, 299))

    # Convert PIL image to numpy array of 3-dimensions
    x = image.img_to_array(img)

    # Add one more dimension
    x = np.expand_dims(x, axis = 0)

    # preprocess images using preprocess_input() from inception module
    y = preprocess_input(x)
    
    image_vector = model_new.predict(y)

    #print the numpy array output flatten layer
    ix = int(val_imglist[i].split('.')[0])
    val_photos[ix] = image_vector.reshape(-1)

In [51]:
from itertools import islice
list(islice(val_descriptions.items(), 5))

[(397133,
  ['startseq man is in kitchen making pizzas endseq',
   'startseq man in apron standing on front of oven with pans and bakeware',
   'startseq baker is working in the kitchen rolling dough endseq',
   'startseq person standing by stove in kitchen endseq',
   'startseq table with pies being made and person standing near wall with pots and pans hanging on the wall endseq']),
 (37777,
  ['startseq the dining table near the kitchen has bowl of fruit on it endseq',
   'startseq small kitchen has various appliances and table endseq',
   'startseq the kitchen is clean and ready for us to see endseq',
   'startseq kitchen and dining area decorated in white endseq',
   'startseq kitchen that has bowl of fruit on the table endseq']),
 (252219,
  ['startseq person with shopping cart on city street',
   'startseq city dwellers walk by as homeless man begs for cash endseq',
   'startseq people walking past homeless man begging on city street',
   'startseq homeless man holding cup and st

In [52]:
val_vocabulary = set()
for key in val_descriptions.keys():
    [val_vocabulary.update(d.split()) for d in val_descriptions[key]]
print('Original val Vocabulary Size: %d' % len(val_vocabulary))

Original val Vocabulary Size: 7004


In [53]:
# convert a dictionary of clean descriptions to a list of descriptions
def to_lines(val_val_descriptions):
    all_desc = list()
    for key in val_descriptions.keys():
        [all_desc.append(d) for d in val_descriptions[key]]
    return all_desc
# calculate the length of the description with the most words
def max_length_val(val_descriptions):
    lines = to_lines(val_descriptions)
    return max(len(d.split()) for d in lines)
# determine the maximum sequence length
max_length_val = max_length_val(val_descriptions)
print('Max Description Val Length: %d' % max_length_val)

Max Description Val Length: 44


In [54]:
ixtoword_val = {}
wordtoix_val = {}
ix = 0
for w in val_vocabulary:
    wordtoix_val[w] = ix
    ixtoword_val[ix] = w
    ix += 1
print(wordtoix_val.items())

dict_items([('onlooker', 0), ('roof', 1), ('nat', 2), ('toast', 3), ('cow', 4), ('seemingly', 5), ('dozen', 6), ('baskest', 7), ('camode', 8), ('thumps', 9), ('cattle', 10), ('wiping', 11), ('purse', 12), ('amazement', 13), ('llama', 14), ('stranding', 15), ('swiftly', 16), ('towel', 17), ('handheld', 18), ('emergencies', 19), ('heavily', 20), ('gestures', 21), ('husband', 22), ('wears', 23), ('crown', 24), ('destroy', 25), ('studying', 26), ('blanketed', 27), ('because', 28), ('spackling', 29), ('deckered', 30), ('bank', 31), ('santa', 32), ('hook', 33), ('informally', 34), ('stalking', 35), ('aluminum', 36), ('rams', 37), ('cherry', 38), ('measuring', 39), ('collectibles', 40), ('mast', 41), ('sunny', 42), ('scooter', 43), ('assembled', 44), ('mountain', 45), ('crossroads', 46), ('chipping', 47), ('lays', 48), ('detroit', 49), ('stormy', 50), ('lifeguarding', 51), ('produce', 52), ('stoplights', 53), ('closet', 54), ('burned', 55), ('listed', 56), ('handicap', 57), ('tip', 58), ('tat

In [55]:
# data generator, intended to be used in a call to model.fit_generator()
def val_generator(val_descriptions, val_photos, wordtoix_val, max_length_val, num_photos_per_batch):
    X1, X2, y = list(), list(), list()
    n=0
    # loop for ever over images
    while 1:
        for key, desc_list in val_descriptions.items():
            n+=1
            # retrieve the photo feature
            photo = val_photos[key]
            for desc in desc_list:
                # encode the sequence
                seq = [wordtoix_val[word] for word in desc.split(' ') if word in wordtoix_val]
                #print(len(seq))
                # split one sequence into multiple X, y pairs
                for i in range(1, len(seq)):
                    # split into input and output pair
                    in_seq, out_seq = seq[:i], seq[i]
                    # pad input sequence
                    in_seq = pad_sequences([in_seq], maxlen=max_length_val)[0]
                    #print("Inseq len: "+ str(len(in_seq)))
                    # encode output sequence
                    out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
                    # store
                    #print("outseq len: "+ str(len(out_seq)))
                    X1.append(photo)
                    X2.append(in_seq)
                    y.append(out_seq)
            # yield the batch data
            if n==num_photos_per_batch:
                yield [[np.array(X1), np.array(X2)], np.array(y)]
                X1, X2, y = list(), list(), list()
                n=0

In [56]:
embedding_dim = 200
vocab_size_val = len(val_vocabulary)
# Get 200-dim dense vector for each of the 10000 words in out vocabulary
embedding_matrix_val = np.zeros((vocab_size, embedding_dim))
for word, i in wordtoix_val.items():
    #if i < max_words:
    embedding_vector_val = embeddings_index.get(word)
    if embedding_vector_val is not None:
        # Words not found in the embedding index will be all zeros
        embedding_matrix_val[i-1] = embedding_vector_val

# Training the Model

In [57]:
# Dont run this
#STEP_SIZE_TRAIN= len(images)/32
#STEP_SIZE_VAL= len(val_images)/32
#num_epochs = 2
#a = model.fit_generator(
#    data_generator(descriptions, photos, wordtoix, max_length=24, num_photos_per_batch=5),
#    steps_per_epoch= STEP_SIZE_TRAIN,
#    validation_data= val_generator(val_descriptions, val_photos, wordtoix_val, max_length_val=24, num_photos_per_batch=5),
#    validation_steps= STEP_SIZE_VAL,
#    epochs= num_epochs)

# Get Blue Score

In [58]:
val_X = np.array(list(val_photos.values()))
val_y = np.array(list(val_descriptions.values()))

In [59]:
from nltk.translate.bleu_score import sentence_bleu,corpus_bleu
def validator(val_X, val_y):
    caption_score=[]
    totalscore = 0.0
    avg_blue_score = 0
    for i in range(0,len(val_photos)):
        val_photo = np.reshape(np.array(list(val_photos.values())[i]),(1,2048))
        result = greedySearch(val_photo)
        ref = []
        list_result=[]
        for desc in val_descriptions[list((val_photos.keys()))[i]]:
            ref.append(desc.split()[1:-1])
            ##print("References")
        list_of_refernces=[ref]
        list_result.append(result.split())
        caption_score.append(corpus_bleu(list_of_refernces,list_result,weights=(1,0,0,0)))
    #calculates caption score for each predicted caption on the test set
    #print(caption_score)
    #finding the average overall score
    avg_blue_score = sum(caption_score)/len(caption_score)
    return avg_blue_score
    #print("Average Score:"+ str(sum(caption_score)/len(caption_score)))

In [60]:
class Metrics(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self._data = []

    def on_epoch_end(self, batch, logs={}):
        X_val, y_val = val_X, val_y

        self._data.append({
            'val_blue': validator(X_val, y_val),
        })
        return

    def get_data(self):
        return self._data

metrics = Metrics()

# Training with early stopping with Blue 1 metric and val loss

In [61]:
def greedySearch(photo):
    in_text = 'startseq'
    max_length=46
    for i in range(max_length):
        sequence = [wordtoix[w] for w in in_text.split() if w in wordtoix]
        sequence = pad_sequences([sequence], maxlen=max_length)
        yhat = model.predict([photo,sequence], verbose=0)
        yhat = np.argmax(yhat)
        word = ixtoword[yhat]
        in_text += ' ' + word
        if word == 'endseq':
            break
    final = in_text.split()
    final = final[1:-1]
    final = ' '.join(final)
    return final

In [62]:
# # save model weights while model is under training
batch_size = 32
STEP_SIZE_TRAIN = len(photos)/batch_size
STEP_SIZE_VAL = len(val_photos)/batch_size
num_epochs =20

checkpoint = ModelCheckpoint(
     'feature-extraction-weights.{epoch:02d}-{val_loss:.2f}.hdf5',
     monitor='val_loss', 
     verbose=1,
     save_best_only=True
     )

es = EarlyStopping(monitor=metrics, mode='min')

blue_threshold = 0.6
class acc_Callback(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}): 
        print(metrics.get_data()[epoch]['val_blue'])
        if(metrics.get_data()[epoch]['val_blue']) > blue_threshold:   
            self.model.stop_training = True
            
my_callback = acc_Callback()

history = model.fit_generator(
    data_generator(descriptions, photos, wordtoix, max_length=46, num_photos_per_batch=5),
    steps_per_epoch= STEP_SIZE_TRAIN,
    validation_data= val_generator(val_descriptions, val_photos, wordtoix_val, 46, num_photos_per_batch=5),
    validation_steps= STEP_SIZE_VAL,
    epochs= num_epochs,
    callbacks = [checkpoint,metrics,my_callback]
 )

Epoch 1/20
1563/1562 [==============================] - 104s 67ms/step - loss: 5.1842 - accuracy: 0.1795 - val_loss: 13.5328 - val_accuracy: 0.0000e+00

Epoch 00001: val_loss improved from inf to 13.53281, saving model to feature-extraction-weights.01-13.53.hdf5


/home/anish_saha/anaconda3/envs/caption/lib/python3.7/site-packages/nltk/translate/bleu_score.py:523: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/anish_saha/anaconda3/envs/caption/lib/python3.7/site-packages/nltk/translate/bleu_score.py:523: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/anish_saha/anaconda3/envs/caption/lib/python3.7/site-packages/nltk/translate/bleu_score.py:523: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it cont

0.39349672735113495
Epoch 2/20
1563/1562 [==============================] - 100s 64ms/step - loss: 4.3352 - accuracy: 0.2490 - val_loss: 14.0220 - val_accuracy: 0.0000e+00

Epoch 00002: val_loss did not improve from 13.53281
0.45752438155147046
Epoch 3/20
1563/1562 [==============================] - 100s 64ms/step - loss: 4.0340 - accuracy: 0.2777 - val_loss: 13.8078 - val_accuracy: 0.0000e+00

Epoch 00003: val_loss did not improve from 13.53281
0.46411397189674125
Epoch 4/20
1563/1562 [==============================] - 99s 63ms/step - loss: 3.9378 - accuracy: 0.2891 - val_loss: 16.1283 - val_accuracy: 0.0000e+00

Epoch 00004: val_loss did not improve from 13.53281
0.4043942974825094
Epoch 5/20
1563/1562 [==============================] - 100s 64ms/step - loss: 3.8302 - accuracy: 0.2982 - val_loss: 14.5819 - val_accuracy: 0.0000e+00

Epoch 00005: val_loss did not improve from 13.53281
0.4243185449414849
Epoch 6/20
1563/1562 [==============================] - 101s 64ms/step - loss: 3.78

In [6]:
new_model = create_model()

NameError: name 'create_model' is not defined

In [4]:
model.save_weights("iter20weights.h5")

NameError: name 'model' is not defined

# Testing

In [67]:
test_data = ids[-5]


In [68]:
test_data

444010